Exploratory Data Analysis (EDA)

Project: NordTech Order Data

This notebook explores the raw NordTech order dataset prior to any permanent cleaning or transformation.
The purpose of this exploratory data analysis is to understand the dataset’s structure, grain, data quality, and potential issues in order to design a robust and well-informed data cleaning and transformation pipeline in the next stage of the project.

Scope

This EDA focuses exclusively on identifying data quality issues, structural properties, and inconsistencies.
Any data type conversions performed in this notebook are temporary and used only for inspection purposes.
All permanent cleaning, normalization, and imputation decisions are deferred to the transformation pipeline.

In this notebook, we:
- Load and inspect the raw dataset
- Examine column data types and missing values
- Identify inconsistent formats (dates, prices, regions, payment methods)
- Check for duplicates and logical inconsistencies
- Document findings that inform the transformation step

Dataset Grain

Each row in the dataset represents a single order line (orderrad) within a customer order.
An order (order_id) may contain multiple order lines, each corresponding to a specific product (produkt_sku) and quantity (antal).

1. Load Raw Dataset

In [20]:
import pandas as pd
import numpy as np

df=pd.read_csv('../data/raw/nordtech_data.csv')

2. Initial Data Inspection

In [2]:
df.head()

,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
0,ORD-2024-00001,ORD-2024-00001-1,2024-05-19,2024-05-22,SKU-WC001,Webbkamera HD,Tillbehör,1,SEK 799,Uppsala,Privat,Kort,KND-53648,Levererad,NaN,NaN,NaN
1,ORD-2024-00002,ORD-2024-00002-1,2024-12-02,5 december 2024,SKU-HB001,USB-C Hub 7-port,Tillbehör,1,549.00,Göteborg,Privat,Swish,KND-84095,Levererad,NaN,NaN,NaN
2,ORD-2024-00003,ORD-2024-00003-1,2024-12-31,2025-01-03,SKU-SD001,Extern SSD 1TB,Lagring,1,1199.00,NaN,Företag,Faktura,KND-91748,Levererad,Stämmer inte överens med produktbeskrivningen.,2025-01-12,2.0
3,ORD-2024-00003,ORD-2024-00003-2,2024-12-31,2025-01-03,SKU-SD002,Extern SSD 500GB,Lagring,10,699 kr,Stockholm,Företag,FAKTURA,KND-91748,Mottagen,"Leveransen tog lite längre än utlovat, men pro...",2025-01-14,3.0
4,ORD-2024-00003,ORD-2024-00003-3,2024-12-31,2025-01-03,SKU-MS001,Trådlös Mus X1,Tillbehör,1,399.00,Stockholm,Företag,Faktura,KND-91748,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2767 entries, 0 to 2766
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         2767 non-null   object 
 1   orderrad_id      2767 non-null   object 
 2   orderdatum       2767 non-null   object 
 3   leveransdatum    2767 non-null   object 
 4   produkt_sku      2767 non-null   object 
 5   produktnamn      2767 non-null   object 
 6   kategori         2767 non-null   object 
 7   antal            2767 non-null   object 
 8   pris_per_enhet   2767 non-null   object 
 9   region           2612 non-null   object 
 10  kundtyp          2767 non-null   object 
 11  betalmetod       2651 non-null   object 
 12  kund_id          2767 non-null   object 
 13  leveransstatus   2673 non-null   object 
 14  recension_text   1355 non-null   object 
 15  recensionsdatum  1355 non-null   object 
 16  betyg            1355 non-null   float64
dtypes: float64(1),

In [4]:
df.shape

(2767, 17)

Finding:
- The dataset contains a mix of numerical, categorical, textual, and date-related fields.
- Several columns that represent dates and prices are stored as object types.
- The dataset includes multiple order lines per order, not one row per order.

Implication:
Data type validation and conversion will be required during transformation.

3. Missing Values Analysis

In [5]:
df.isna().sum()

order_id              0
orderrad_id           0
orderdatum            0
leveransdatum         0
produkt_sku           0
produktnamn           0
kategori              0
antal                 0
pris_per_enhet        0
region              155
kundtyp               0
betalmetod          116
kund_id               0
leveransstatus       94
recension_text     1412
recensionsdatum    1412
betyg              1412
dtype: int64

Finding:
Missing values are present across multiple columns, including both numerical and categorical features.

Implication:
A structured and type-aware missing value handling strategy is required.

In [6]:
# Column Summary Table
summary = pd.DataFrame({
    "dtype": df.dtypes,
    "missing": df.isna().sum(),
    "unique": df.nunique()
})
summary

,dtype,missing,unique
order_id,object,0,1657
orderrad_id,object,0,2700
orderdatum,object,0,536
leveransdatum,object,0,544
produkt_sku,object,0,17
produktnamn,object,0,17
kategori,object,0,5
antal,object,0,22
pris_per_enhet,object,0,76
region,object,155,36


4. Date Columns Investigation

In [7]:
date_cols = ["orderdatum", "leveransdatum", "recensionsdatum"]

for col in date_cols:
    print(f"\nUnique samples from {col}:")
    print(df[col].dropna().astype(str).unique()[:20])


Unique samples from orderdatum:
['2024-05-19' '2024-12-02' '2024-12-31' '2024-04-22' '2024-07-01'
 '2024-03-10' '2024-06-16' '2024-08-07' '2024-06-10' '2024/06/10'
 '2024-10-26' '2024-07-26' '2024-04-08' '2024-02-23' '2024-05-14'
 '2024-11-25' '2024-04-27' '2024-10-24' '2024-12-05' '2024-06-20']

Unique samples from leveransdatum:
['2024-05-22' '5 december 2024' '2025-01-03' '2024-04-26' '2024-07-05'
 '2024-03-12' '2024-06-20' '2024-08-14' '2024-06-12' '2024-10-30'
 '2024-07-28' '2024-04-11' '2024-02-25' '2024-05-16' '2024-11-27'
 '2024-05-01' '2024/10/27' '2024-12-07' '2024-06-22' '2024-11-02']

Unique samples from recensionsdatum:
['2025-01-12' '2025-01-14' '2024-07-12' '2024-03-19' '2024-08-24'
 '2024-08-21' '2024-06-15' '2024-05-23' '7 maj 2024' '2024-12-08'
 '2024-12-16' '2024-07-06' '2024-07-21' '2024-12-09' '2024-03-17'
 '2024-09-26' '2024-11-08' '2024-02-15' '2024-02-05' '2024-08-30']


Finding:
- Multiple date formats exist across all date columns.
- Some values cannot be parsed directly.
- Some delivery dates occur before order dates.

Implication:
A robust date-parsing function and logical validation rules are required during transformation.

5. Price Format Inspection

In [9]:
df["pris_per_enhet"].astype(str).unique()[:30]

array(['SEK 799', '549.00', '1199.00', '699 kr', '399.00', '799.00',
       '1899.00', '599.00', '4999.00', '5999.00', '18999.00', '1299.00',
       '14999.00', '699.00', '2499.00', '7999.00', '499.00', '4999 kr',
       '599 kr', '899.00', '899 kr', '799:-', '18999 kr', '1 899,00',
       '7999 kr', '399 kr', '599:-', 'SEK 1899', '549 kr', '1299:-'],
      dtype=object)

Finding:
Prices are stored in inconsistent textual formats (currency symbols, spacing, and decimal separators),
e.g. "SEK 799", "699 kr", "599:-", "1 899,00".

Implication:
Price values must be standardized and converted to a numerical format before analysis.

6. Region Values Inspection

In [17]:
df["region"].unique()

array(['Uppsala', 'Göteborg', nan, 'Stockholm', 'Örebro', 'örebro',
       'Orebro', 'Norrland', 'Linköping', 'Malmö', 'Västerås', 'GÖTEBORG',
       'LINKÖPING', 'stockholm', 'Gothenburg', 'STHLM', 'STOCKHOLM',
       'malmo', 'linköping', 'uppsala', 'Sthlm', 'GBGB', 'västerås',
       'MALMÖ', 'Sthml', 'Gbg', 'NORRLAND', 'ÖREBRO', 'norrland', 'Norr',
       'Vasteras', 'UPPSALA', 'göteborg', 'Malmo', 'Linkoping',
       'VÄSTERÅS', 'malmö'], dtype=object)

Finding:
Region names are inconsistently formatted (case differences and abbreviations).

Implication:
Region values must be standardized during data cleaning to ensure correct aggregation and reporting.

7. Payment Method Inspection

In [21]:
df["betalmetod"].unique()

array(['Kort', 'Swish', 'Faktura', 'FAKTURA', 'Mobilbetalning', 'faktura',
       'KORT', 'SWISH', 'Kreditkort', 'swish', nan, 'Invoice', 'Visa',
       'kort', 'Mastercard'], dtype=object)

Finding:
Payment methods are inconsistently labeled due to casing differences and naming variations.

Implication:
Payment methods require standardization to avoid duplicated categories.

8. Delivery Status Inspection

In [23]:
df["leveransstatus"].unique()

array(['Levererad', 'Mottagen', nan, 'LEVERERAD', 'Retur', 'retur',
       'Under transport', 'Skickad', 'Returnerad', 'levererad',
       'Återsänd', 'På väg', 'under transport', 'RETUR',
       'UNDER TRANSPORT'], dtype=object)

Finding:
The delivery status (leveransstatus) column contains multiple inconsistent representations, including:
- Case variations (e.g. "Levererad", "LEVERERAD", "levererad")
- Multiple labels representing similar outcomes (e.g. "Retur", "Returnerad", "Återsänd")
- Multiple labels for in-transit states (e.g. "Under transport", "UNDER TRANSPORT", "På väg", "Skickad")
- Missing values

Implication:
Delivery status values must be standardized into a defined set of categories
(e.g. Delivered, In Transit, Returned) during the transformation stage to enable
accurate delivery performance and return-rate analysis.

9. Antal (Quantity) Analysis

In [29]:
df["antal"].unique()

array(['1', '10', '1 st', '3', '2', '5', '2st', '"2"', '2 st', '1st',
       'en', '"1"', 'två', '3st', '"3"', '5 st', 'tre', '10 st', '3 st',
       '"5"', '5st', 'fem'], dtype=object)

In [30]:
df["antal"].describe()

count     2767
unique      22
top          1
freq      1839
Name: antal, dtype: object

Finding:
The quantity (antal) column contains highly inconsistent formats, including:
- Numeric values stored as strings (e.g. "1", "2", "10")
- Values with unit annotations (e.g. "1 st", "2st", "10 st")
- Numbers enclosed in quotation marks (e.g. '"2"')
- Swedish number words (e.g. "en", "två", "tre", "fem")
- Mixed formats combining numbers and text (e.g. "5st")

Implication:
The antal column cannot be directly converted to a numeric type.
A custom parsing and normalization strategy is required during the transformation. 

10. Kundtyp (Customer Type) Analysis

In [32]:
df["kundtyp"].unique()

array(['Privat', 'Företag', 'FÖRETAG', 'PRIVAT', 'B2C', 'privat',
       'företag', 'B2B', 'b2c', 'Konsument', 'Firma', 'b2b'], dtype=object)

Finding:
The customer type (kundtyp) column contains multiple representations of the same categories, including:
- Case variations (e.g. "Privat", "PRIVAT", "privat")
- Language variations (e.g. "Privat", "Konsument")
- Business terminology (e.g. "Företag", "Firma")
- Business model labels (e.g. "B2B", "B2C")

Implication:
Customer types must be normalized into a small, well-defined set of categories
(e.g. B2C and B2B) during transformation to ensure reliable segmentation and analysis.

11. Betyg (Customer Rating) Analysis

In [34]:
df["betyg"].unique()

array([nan,  2.,  3.,  4.,  1.,  5.])

In [35]:
df["betyg"].isna().sum()

np.int64(1412)

In [36]:
df["betyg"].value_counts(dropna=False)

betyg
NaN    1412
4.0     429
5.0     425
3.0     258
1.0     132
2.0     111
Name: count, dtype: int64

Finding:
- Ratings are bounded between 1 and 5.
- The distribution is slightly skewed toward higher ratings (common in e‑commerce).
- Missing values are relatively few.
- Using the mean could introduce decimal values (e.g., 3.67), which do not naturally occur in rating systems.
- Using the mode (most common value) would over‑represent the most frequent rating and distort the distribution.

Decision: Use the Median
The median is the most robust and statistically appropriate choice for ordinal ratings because:
- It is not affected by skewed distributions.
- It avoids unrealistic decimal values.
- It preserves the central tendency of the data.
- It prevents artificially inflating or deflating the overall rating score.

12. Duplicate Records Check

In [47]:
df.duplicated().sum()

np.int64(67)

In [48]:
df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist())

,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
107,ORD-2024-00070,ORD-2024-00070-1,2024-11-18,2024-11-20,SKU-KB002,Kompakt Tangentbord Mini,Tillbehör,1,599.00,Stockholm,Privat,Kort,KND-13904,Levererad,Riktigt nöjd! Använder den dagligen.,2024-11-23,4.0
613,ORD-2024-00070,ORD-2024-00070-1,2024-11-18,2024-11-20,SKU-KB002,Kompakt Tangentbord Mini,Tillbehör,1,599.00,Stockholm,Privat,Kort,KND-13904,Levererad,Riktigt nöjd! Använder den dagligen.,2024-11-23,4.0
181,ORD-2024-00110,ORD-2024-00110-1,2024-07-11,2024-07-14,SKU-MS001,Trådlös Mus X1,Tillbehör,2,399 kr,Gbg,privat,Kort,KND-13544,Levererad,"Leverans på två dagar, imponerad!",2024-07-21,4.0
1403,ORD-2024-00110,ORD-2024-00110-1,2024-07-11,2024-07-14,SKU-MS001,Trådlös Mus X1,Tillbehör,2,399 kr,Gbg,privat,Kort,KND-13544,Levererad,"Leverans på två dagar, imponerad!",2024-07-21,4.0
182,ORD-2024-00111,ORD-2024-00111-1,2024-06-21,2024-06-26,SKU-MN002,"Bildskärm 24"" FHD",Bildskärmar,1,2499.00,NORRLAND,Företag,Faktura,KND-39076,Levererad,Leveransskada - kartongen var helt demolerad.,2024-07-06,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,ORD-2024-01625,ORD-2024-01625-2,2024-11-25,2024-11-28,SKU-WC001,Webbkamera HD,Tillbehör,1st,799.00,STOCKHOLM,Företag,Faktura,KND-26611,Levererad,Fantastisk produkt! Fungerar precis som utlovat.,2024-12-04,5.0
2489,ORD-2024-01625,ORD-2024-01625-2,2024-11-25,2024-11-28,SKU-WC001,Webbkamera HD,Tillbehör,1st,799.00,STOCKHOLM,Företag,Faktura,KND-26611,Levererad,Fantastisk produkt! Fungerar precis som utlovat.,2024-12-04,5.0
2713,ORD-2024-01625,ORD-2024-01625-2,2024-11-25,2024-11-28,SKU-WC001,Webbkamera HD,Tillbehör,1st,799.00,STOCKHOLM,Företag,Faktura,KND-26611,Levererad,Fantastisk produkt! Fungerar precis som utlovat.,2024-12-04,5.0
1377,ORD-2024-01634,ORD-2024-01634-1,2024-12-28,2025-01-01,SKU-WC001,Webbkamera HD,Tillbehör,1,799.00,Linköping,B2C,swish,KND-46423,Levererad,NaN,NaN,NaN


In [50]:
for col in ["order_id", "orderrad_id", "kund_id", "produkt_sku"]:
    print(col, df[col].nunique(), "/", len(df))


order_id 1657 / 2767
orderrad_id 2700 / 2767
kund_id 1644 / 2767
produkt_sku 17 / 2767


Finding:

ORD-2024-00070   ORD-2024-00070-1   (same row repeated twice)
ORD-2024-00110   ORD-2024-00110-1   (same row repeated twice)
ORD-2024-01625   ORD-2024-01625-2   (same row repeated 3 times)
ORD-2024-01634   ORD-2024-01634-1   (same row repeated twice)

67 full-row duplicates were identified.
These records appear to be repeated entries with identical values across all columns, indicating duplicated ingestion rather than distinct order events.
unique key=total - NaN = 2767 - 67 = 2700 so orderrad_id is unique key

Implication:
Duplicate handling rules must be defined during the transformation stage.

13. Logical Consistency Check (Delivery vs Order Date)

In [52]:
# Delivery earlier than order
mask = pd.to_datetime(df["leveransdatum"], errors="coerce") < pd.to_datetime(df["orderdatum"], errors="coerce")
df[mask].head()

,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
54,ORD-2024-00037,ORD-2024-00037-1,2024-02-17,2024-02-14,SKU-SD002,Extern SSD 500GB,Lagring,1,699.00,Göteborg,Privat,Kreditkort,KND-35320,Retur,NaN,NaN,NaN
59,ORD-2024-00038,ORD-2024-00038-5,2024-11-01,2024-10-31,SKU-LP002,Laptop Budget 14,Datorer,2,7999.00,Göteborg,Företag,Faktura,KND-76077,Levererad,NaN,NaN,NaN
68,ORD-2024-00044,ORD-2024-00044-2,2024-10-15,2024-10-10,SKU-WC001,Webbkamera HD,Tillbehör,1,799.00,Linköping,Privat,Faktura,KND-75717,Levererad,NaN,NaN,NaN
112,ORD-2024-00072,ORD-2024-00072-2,2024-04-13,2024-04-11,SKU-MS002,Ergonomisk Mus Pro,Tillbehör,1,699.00,Uppsala,Konsument,Kort,KND-81573,Levererad,NaN,NaN,NaN
143,ORD-2024-00093,ORD-2024-00093-1,2024-04-26,2024-04-22,SKU-KB001,Mekaniskt Tangentbord K7,Tillbehör,2,1299.00,MALMÖ,Företag,Kort,KND-29299,Levererad,Mycket nöjd med köpet. Snabb leverans också!,13/05/2024,5.0


Finding:
Some records indicate delivery dates occurring before order dates.

Implication:
Logical validation rules are required to detect and handle invalid temporal relationships.

14. Recension_text analysis

In [54]:
df["recension_text"].head()

0                                                  NaN
1                                                  NaN
2       Stämmer inte överens med produktbeskrivningen.
3    Leveransen tog lite längre än utlovat, men pro...
4                                                  NaN
Name: recension_text, dtype: object

In [55]:
df["recension_text"].isna().sum()

np.int64(1412)

In [56]:
df["recension_text"].str.len().describe()

count    1355.000000
mean       40.651661
std         8.194442
min        23.000000
25%        36.000000
50%        40.000000
75%        44.000000
max        64.000000
Name: recension_text, dtype: float64

14. 1. Leading/trailing whitespace

In [57]:
(df["recension_text"].astype(str) != df["recension_text"].astype(str).str.strip()).sum()


np.int64(0)

14. 2. Empty strings

In [58]:
(df["recension_text"].astype(str).str.strip() == "").sum()

np.int64(0)

14. 3. check other placeholders

In [59]:
df["recension_text"].astype(str).str.strip().str.lower().isin(
    ["nan", "none", "null", "missing"]
).sum()

np.int64(1412)

Finding:
- No leading or trailing whitespace was detected.
- No empty review strings were found.
- A large number of entries evaluate to the literal string "nan" when converted to text, indicating the presence of either true missing values or textual placeholders representing missing data.

Implication:
These values must be consistently converted to proper missing values (NaN) during transformation to prevent placeholder text from being treated as valid review content.

15. ID Columns Quality Check (order_id, orderrad_id, kund_id, produkt_sku)

In [61]:
id_cols = ["order_id", "orderrad_id", "kund_id", "produkt_sku"]

for col in id_cols:
    if col in df.columns:
        print(f"\nColumn: {col}")
        print("  Missing values:", df[col].isna().sum())
        print("  Unique values:", df[col].nunique())
        print("  Sample values:", df[col].astype(str).unique()[:15])


Column: order_id
  Missing values: 0
  Unique values: 1657
  Sample values: ['ORD-2024-00001' 'ORD-2024-00002' 'ORD-2024-00003' 'ORD-2024-00004'
 'ORD-2024-00005' 'ORD-2024-00006' 'ORD-2024-00007' 'ORD-2024-00008'
 'ORD-2024-00009' 'ORD-2024-00010' 'ORD-2024-00011' 'ORD-2024-00012'
 'ORD-2024-00013' 'ORD-2024-00014' 'ORD-2024-00015']

Column: orderrad_id
  Missing values: 0
  Unique values: 2700
  Sample values: ['ORD-2024-00001-1' 'ORD-2024-00002-1' 'ORD-2024-00003-1'
 'ORD-2024-00003-2' 'ORD-2024-00003-3' 'ORD-2024-00004-1'
 'ORD-2024-00005-1' 'ORD-2024-00005-2' 'ORD-2024-00005-3'
 'ORD-2024-00005-4' 'ORD-2024-00006-1' 'ORD-2024-00007-1'
 'ORD-2024-00008-1' 'ORD-2024-00008-2' 'ORD-2024-00009-1']

Column: kund_id
  Missing values: 0
  Unique values: 1644
  Sample values: ['KND-53648' 'KND-84095' 'KND-91748' 'KND-95459' 'KND-32599' 'KND-60466'
 'KND-69810' 'KND-76965' 'KND-87518' 'KND-41816' 'KND-65290' 'KND-55095'
 'KND-21853' 'KND-21867' 'KND-90244']

Column: produkt_sku
  Missing v

Finding:
All ID columns (order_id, orderrad_id, kund_id, produkt_sku) are structurally clean.
- No missing values were detected.
- No leading/trailing whitespace or "nan" strings were observed.
- All IDs follow consistent formatting and behave as stable identifiers.
- order_id is not unique, which is expected because each order can contain multiple order lines.
- orderrad_id is unique, confirming it is the correct primary key for order lines.

Implication:
Although the current dataset does not show ID formatting issues, ID normalization will still be included in the transformation pipeline to ensure robustness and prevent future data quality problems if new raw data is added.

16. EDA Summary

The exploratory analysis identified several data quality issues that must be addressed before modeling or KPI analysis:

Data Type Issues
- Mixed and inconsistent date formats
- Price values stored as text
- Quantity values requiring parsing

Categorical Inconsistencies
- Regions
- Payment methods
- Customer types
- Delivery status

Missing Values
- Present across multiple columns
- Require type-aware imputation strategies

Duplicates
- 67 full-row duplicates
- Safe to remove

Logical Errors
- Delivery dates earlier than order dates

Conclusion
These findings directly inform the design of the data cleaning and transformation pipeline implemented in the next stage of the project.
